In [1]:
key_id = 'PK04UHV69AF2QULV4REU'
secret_key = '7g1qUN7qjsfW3U6oSccYYtHyQdoewJ12ANDQmSKd'
endpoint = 'https://paper-api.alpaca.markets'

In [8]:
from stock_1 import stock
from stock_env import StocksEnv
from brain_DQN import DeepQNetwork
import pandas as pd
import tensorflow.compat.v1 as tf


In [9]:
# connect to alpaca api
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime


stock_client = StockHistoricalDataClient(key_id, secret_key)

# get 1 years worth data on Nvidia by the day. Could also use TimeFrame.Minute for daily data
request_params = StockBarsRequest(symbol_or_symbols=['NVDA'],
                                  timeframe=TimeFrame.Day,
                                  start=datetime(2020, 1, 1),
                                  end=datetime(2022, 1, 1)
                                  )

bars = stock_client.get_stock_bars(request_params)

In [10]:
data=bars.df.reset_index()
data.head()

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,NVDA,2020-01-02 05:00:00+00:00,238.85,239.91,236.720,239.91,5941969.0,67912.0,238.723319
1,NVDA,2020-01-03 05:00:00+00:00,235.00,237.83,234.100,236.07,5145208.0,63605.0,236.384392
2,NVDA,2020-01-06 05:00:00+00:00,232.36,237.27,231.270,237.06,6572812.0,73912.0,235.468778
3,NVDA,2020-01-07 05:00:00+00:00,238.20,241.77,236.390,239.93,7980144.0,81624.0,239.855220
4,NVDA,2020-01-08 05:00:00+00:00,239.76,242.04,238.149,240.38,6931017.0,70240.0,240.313148


In [11]:
data = data.sort_values('timestamp', ascending=True)
data

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,NVDA,2020-01-02 05:00:00+00:00,238.85,239.910,236.7200,239.91,5941969.0,67912.0,238.723319
1,NVDA,2020-01-03 05:00:00+00:00,235.00,237.830,234.1000,236.07,5145208.0,63605.0,236.384392
2,NVDA,2020-01-06 05:00:00+00:00,232.36,237.270,231.2700,237.06,6572812.0,73912.0,235.468778
3,NVDA,2020-01-07 05:00:00+00:00,238.20,241.770,236.3900,239.93,7980144.0,81624.0,239.855220
4,NVDA,2020-01-08 05:00:00+00:00,239.76,242.040,238.1490,240.38,6931017.0,70240.0,240.313148
...,...,...,...,...,...,...,...,...,...
500,NVDA,2021-12-27 05:00:00+00:00,296.40,310.865,296.4000,309.45,40331037.0,594185.0,307.225490
501,NVDA,2021-12-28 05:00:00+00:00,313.11,313.300,300.1181,303.22,42214335.0,629167.0,304.595486
502,NVDA,2021-12-29 05:00:00+00:00,303.11,305.480,293.6600,300.01,34308945.0,495749.0,298.631331
503,NVDA,2021-12-30 05:00:00+00:00,298.24,304.570,295.4000,295.86,30885650.0,408098.0,300.514442


In [13]:
def game_step(observation, step=None, train=True, show_log=False):
    
    # RL choose action based on observation
    action = RL.choose_action(observation, train)

    # RL take action and get next observation and reward
    observation_, reward, done = env.step(action, show_log)

    RL.store_transition(observation, action, reward, observation_)
    # print("total profit:%.3f" % env.total_profit, end='\r')
    if step and (step > 200) and (step % 5 == 0):
        RL.learn()

    # swap observation
    observation = observation_
    
    return observation, done
    

def run(max_round):
    step = 0
    for episode in range(max_round):
        # initial observation
        observation = env.reset()

        while True:
            
            observation, done = game_step(observation, step=step)
            # print(observation)
            # break while loop when end of this episode
            if done:
                break
            step += 1
        
        print('epoch:%d, total_profit:%.3f' % (episode, env.total_profit))
        # BackTest(False)


def BackTest(env, show_log=True):
    observation = env.reset()
    step=0
    while True:
        observation, done = game_step(observation, train=False, show_log=show_log)
        # break while loop when end of this episode
        if done:
            break
    print('total_profit:%.3f' % (env.total_profit))
    return env




In [16]:
if __name__ == "__main__":
    max_round = 50
 
    data = data.sort_values('timestamp', ascending=True)
    trend = data.close.values.tolist() #选取收盘数据做测试
    env = stock(trend[0:1750])
    RL = DeepQNetwork(env.n_actions, env.n_features,
                      learning_rate=0.01,
                      reward_decay=0.9,
                      e_greedy=0.9,
                      replace_target_iter=200,
                      memory_size=3000,
                      # output_graph=True
                      )
    
    run(max_round)
    env = stock(trend[1750:])
    env = BackTest(env, show_log=False)
    env.draw()

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-10-09 15:37:05.396223: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-09 15:37:05.396710: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-10-09 15:37:05.402635: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-10-09 15:37:05.408877: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-09 15:37:05.415846: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-09 15:37:05.461084: I tensorflow/core/grappler/optimizers/custom_graph

epoch:0, total_profit:0.000
epoch:1, total_profit:0.000
epoch:2, total_profit:0.000
epoch:3, total_profit:0.000
epoch:4, total_profit:0.000
epoch:5, total_profit:0.000
epoch:6, total_profit:0.000
epoch:7, total_profit:0.000
epoch:8, total_profit:0.000
epoch:9, total_profit:0.000
